In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import os
# os.chdir('/content/drive/My Drive/Colab Notebooks/MachineLearningPractice/FinanceRiskControl')
# !ls

In [15]:
import numpy as np
np.random.seed(10)
 
import matplotlib.pyplot as plt
import xgboost as xgb
import pandas as pd
 
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ( RandomForestClassifier,
                              GradientBoostingClassifier)
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve,  roc_auc_score
from scipy.sparse import hstack
from sklearn.model_selection import StratifiedKFold, KFold

In [16]:
X, y = make_classification(n_samples=80000)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)
 
# It is important to train the ensemble of trees on a different subset
# of the training data than the linear regression model to avoid
# overfitting, in particular if the total number of leaves is
# similar to the number of training samples

X_train, X_train_lr, y_train, y_train_lr = train_test_split(
    X_train, y_train, test_size=0.5)
 
n_estimator = 10

In [17]:
X_train.shape

(20000, 20)

In [38]:
type(X_train)

numpy.ndarray

In [39]:
X_train

array([[-0.14232498,  0.40691798, -0.25310425, ..., -0.73100051,
        -1.28398293, -0.68876362],
       [ 0.01510832,  0.0828175 , -0.60199552, ...,  0.39689891,
         1.31079358,  0.15905713],
       [-0.71718941, -0.39409991,  0.430894  , ...,  0.89911773,
        -0.83384519,  0.44686277],
       ...,
       [ 1.15680052,  0.47725931,  0.53351541, ..., -0.3211597 ,
        -0.31467868, -0.22677577],
       [ 0.52343559, -0.24572346, -0.50340715, ..., -1.5816383 ,
        -0.87050009,  0.74326295],
       [ 0.55248371,  0.33979366,  0.5671295 , ..., -0.97899241,
        -0.10280345,  0.24605206]])

In [36]:
xgb_cls = xgb.XGBClassifier(nthread=2,     #含义：nthread=-1时，使用全部CPU进行并行运算（默认）, nthread=1时，使用1个CPU进行运算。
                          learning_rate=0.08,    #含义：学习率，控制每次迭代更新权重时的步长，默认0.3。调参：值越小，训练越慢。典型值为0.01-0.2。
                          n_estimators=50,       #含义：总共迭代的次数，即决策树的个数
                          max_depth=5,           #含义：树的深度，默认值为6，典型值3-10。调参：值越大，越容易过拟合；值越小，越容易欠拟合
                          gamma=0,               #含义：惩罚项系数，指定节点分裂所需的最小损失函数下降值。
                          subsample=0.9,       #含义：训练每棵树时，使用的数据占全部训练集的比例。默认值为1，典型值为0.5-1。调参：防止overfitting。
                          colsample_bytree=0.5) #训练每棵树时，使用的特征占全部特征的比例。默认值为1，典型值为0.5-1。调参：防止overfitting。 
xgb_cls.fit(X_train, y_train)

xgb_enc = OneHotEncoder()
xgb_enc.fit(xgb_cls.apply(X_train))

xgb_lm = LogisticRegression(solver='lbfgs', max_iter=1000)
xgb_lm.fit(
    xgb_enc.transform(xgb_cls.apply(X_train_lr)), y_train_lr
)
 
y_pred_xgb_lm = xgb_lm.predict_proba(
    xgb_enc.transform(xgb_cls.apply(X_test)))[:, 1]
fpr_xgb_lm, tpr_xgb_lm, _ = roc_curve(y_test, y_pred_xgb_lm)


In [37]:
type(X_train)

numpy.ndarray

In [24]:
hehe = xgb_cls.apply(X_train)

In [25]:
hehe.shape

(20000, 50)

Return the predicted leaf every tree for each sample.

        Parameters
        ----------
        X : array_like, shape=[n_samples, n_features]
            Input features matrix.

        ntree_limit : int
            Limit number of trees in the prediction; defaults to 0 (use all trees).

        Returns
        -------
        X_leaves : array_like, shape=[n_samples, n_trees]
            For each datapoint x in X and for each tree, return the index of the
            leaf x ends up in. Leaves are numbered within
            ``[0; 2**(self.max_depth+1))``, possibly with gaps in the numbering.
        

In [26]:
test_encoder = OneHotEncoder()

In [28]:
encoding = test_encoder.fit(hehe)

In [21]:
X_train_ext = hstack([xgb_enc.transform(xgb_cls.apply(X_train_lr)), X_train_lr])
X_test_ext = hstack([xgb_enc.transform(xgb_cls.apply(X_test)), X_test])
xgb_lm.fit(X_train_ext, y_train_lr)
 
y_pred_xgb_originalfeature_lm = xgb_lm.predict_proba(X_test_ext)[:, 1]
fpr_xgb_originalfeature_lm, tpr_xgb_originalfeature_lm, _ = roc_curve(y_test, y_pred_xgb_originalfeature_lm)
print("xgboost新特征与原始特征+LR的AUC为：", roc_auc_score(y_test, y_pred_xgb_originalfeature_lm))

xgboost新特征与原始特征+LR的AUC为： 0.9928625855721991
